In [1]:
## Project Eagle Script by Shean Mobed
#Imports
import pandas as pd
import numpy as np
from glob import glob
import os
import re


#MultiFile
multifile = sorted(glob('Multifile/*_Genotyping Result_*.csv'))
multifile

#Concat Files with same cols
df = pd.concat((pd.read_csv(file, comment = '#').assign(filename = file)
          for file in multifile), ignore_index=True)
#Date Extraction
File1 = df.filename.get(0)
with open (File1) as GenotypingResults:
    for line in GenotypingResults:
        if line[0:13] == '# Exported On': 
            date = line[15:47].replace(':','_').replace('\n','')
            
date2 = date[:10]

date3 = date[:17].replace('-','').replace(' ','').replace('_','')

#File name
basename = os.path.splitext(os.path.basename(File1))[0]
eagle = basename.split('_')[0]
print(eagle)
print(date)
print(date2)
print(date3)

GLS-GP-004274
2021-08-28 10_16_48 AM BST
2021-08-28
202108281016


In [2]:
##Mutaion Caller
#Simplify Table
df = df[['Sample','Call','SNP Assay','Confidence']]

#Sample0

sample0 = df[(df['Sample'] == '0') | 
             (df['Sample'] == 'PosCon') | 
             (df['Sample'] == 'NTC') | 
             (df['Sample'] == 'PHE') |
             (df['Sample'] == 'Empty')
            ]


df = df.drop(sample0.index)


#Blank Sample
df['Sample'].replace('',np.nan, inplace = True)
df.dropna(subset = ['Sample'], inplace = True)

#Rename Cols
df.rename(columns={'SNP Assay': 'CH1-Target',
         'Call': 'CH1-Result',
          'Confidence': 'CH1-Conf'},
         inplace=True)
#Unknown
unknown = df.loc[df.Sample.str.contains("Unknown",case=False)]
df = df.drop(unknown.index)

In [29]:
## Duplicates Troubleshooter

#Duplicate Barcodes error Raise
bcdups = df.groupby(['Sample']).size() > 4
bcdups = pd.DataFrame(bcdups)
bcdups = bcdups.reset_index()
bcdups = bcdups.rename(columns = { 0:'Bool'})
bcdups = bcdups[(bcdups['Bool']==True)]
bcdups = bcdups.loc[bcdups.Bool == True]

#print(bcdups.Sample.count())
bcdups.to_csv('dups.csv')

try:
    if bcdups['Sample'].count() > 0 :
        raise Exception
except Exception:
    print("Check for duplicate barcodes in files!, duplicates can be found in dups.csv")
else: print('No duplicate barcodes :)')

#SNP assay column duplicates error Raise

snpdups = df.groupby(['CH1-Target']).size()
snpdups = pd.DataFrame(snpdups)
snpdups = snpdups.reset_index()
snpdups = snpdups.rename(columns = { 0:'Count'})
samplecount = df.index.size / 4

try:
    if snpdups['Count'][0] > samplecount :
        raise Exception
except Exception:
    print("Check for error in SNP assay column in the files!")
else: print('No error found in SNP assay column')

No duplicate barcodes :)
No error found in SNP assay column
